## Añadir descripcion de proyecto ##

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.shape

(3214, 5)

In [5]:
df[['calls','messages']] = df[['calls','messages']].astype('int')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


Dado que los datos se identificaron de inmediato como los valores previamente procesados, se identificaron considerando el tipo de usuario y cual es el que usa cada tipo de plan (Smart y Ultra), es por ello, que se decidio utilizar la columna *'is_ultra'* que contiene valores booleanos y que estan previamente clasificados,  con la cual podemos trabajar con modelos de clasificación, a continuación se emplean

**RandomForestClassifier**

In [21]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

#Dividir el dataset original en conjuntos de entrenamiento-validacion y prueba, en el mismo se considera el 20% en tamaño de prueba
features_train_val, features_test, target_train_val, target_test = train_test_split(features, target, test_size=0.20, random_state=54321)

#Despues, dividimos el conjunto construido en la primera linea, entrenamiento y validacion
# Para este caso se tiene el 80% del original, y queremos que la validación sea el 25% de eso lo que corresponde a (0.25 * 0.80 = 0.20 del total)

features_train, features_valid, target_train, target_valid = train_test_split(features_train_val, target_train_val, test_size=0.25, random_state=54321)

# Entrenamos el modelo
model = RandomForestClassifier(random_state=54321, n_estimators=50, max_depth=20)
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento

# Calculamos la puntuación de accuracy en el conjunto de validación
score_valid = model.score(features_valid, target_valid)
print(f'Exactitud del Bosque Aleatorio (Validación): {score_valid:.4f}')

# Calcular la puntuación de accuracy en el conjunto de PRUEBA
# Este es el paso final de evaluación, el modelo NO ha visto estos datos en entrenamiento ni validación
test_predictions = model.predict(features_test)
score_test = accuracy_score(target_test, test_predictions)
print(f'Exactitud del Bosque Aleatorio (Prueba): {score_test:.4f}')

Exactitud del Bosque Aleatorio (Validación): 0.8258
Exactitud del Bosque Aleatorio (Prueba): 0.7854


**Prueba de cordura, Clasificador Dummy** <br>
Aplicada al modelo RandomForest, considerado como el mejor modelo con base en los resultados obtenidos<br>
(Esta es la estrategia más común para una prueba de cordura en clasificación.)<br>

Un clasificador ficticio es un modelo simple de aprendizaje automático que realiza predicciones mediante reglas básicas, sin aprender realmente de los datos de entrada.<br> 
Sirve como referencia para comparar el rendimiento de modelos más complejos.<br> 
El clasificador ficticio nos ayuda a comprender si nuestros modelos sofisticados realmente están aprendiendo patrones útiles o simplemente adivinando.<br>
El DummyClassifier aprenderá cuál es la clase más común en el conjunto de entrenamiento y luego predecirá esa clase para todos los ejemplos, sin importar las características.

In [26]:
strategy= 'most_frequent' #hará que siempre prediga la clase que aparece más a menudo en el conjunto de entrenamiento.
dummy_model = DummyClassifier(strategy='most_frequent', random_state=54321)

# Entrenar el clasificador dummy (solo para que aprenda cuál es la clase más frecuente)
dummy_model.fit(features_train, target_train)

# Hacer predicciones con el clasificador dummy en el conjunto de prueba
dummy_predictions = dummy_model.predict(features_test)

# Calcular la precisión del clasificador dummy en el conjunto de prueba
dummy_score = accuracy_score(target_test, dummy_predictions)

print(f'El nivel de precision aplicando la prueba de cordura DummyClassifier para el modelo RandomForest es de: {dummy_score:.4f}')
#print("\n Resultados de la Prueba de Cordura (Dummy Classifier)")
#print(f'Exactitud del Clasificador Dummy (Prueba):', dummy_score)

# Conclusión de la prueba de cordura
#if score_test > dummy_score:
 #   print("\nEl modelo de RandomForest es MEJOR que el clasificador dummy")
  #  print("Esto es una buena señal: el modelo está aprendiendo algo significativo.")
#else:
 #   print("\n¡ALERTA! El modelo RainForest NO es mejor que el clasificador dummy.")
  #  print("Revisa tus datos, tu preprocesamiento o la configuración de tu modelo.")

El nivel de precision aplicando la prueba de cordura DummyClassifier para el modelo RandomForest es de: 0.6641


**DecisionTreeClassifier**

In [24]:
features = df.drop(['is_ultra'],axis=1)
target = df['is_ultra']

#Dividimos en conjuntos
features_train_val, features_test_dt, target_train_val, target_test_dt = train_test_split(features, target, random_state=54321,test_size=0.20)
features_train, features_valid, target_train, target_valid = train_test_split(features_train_val, target_train_val, random_state=54321,test_size=0.25)

#Entrenamos el modelo
model_dtree = DecisionTreeClassifier(random_state = 12345, max_depth =8)
model_dtree.fit(features_train, target_train)

#Calculamos la exactitud en el conjunto de validacion
dtree_score_valid = model_dtree.score(features_valid, target_valid)
print(f'Precision del Arbol de Decision (Validacion): {dtree_score_valid:.4f}')

#Calculamos la exactitud en el modelo de prueba
dtree_predictions = model_dtree.predict(features_test_dt)
dtree_score = accuracy_score(target_test_dt, test_predictions)
print(f'Precision del Arbol de Decision (Prueba): {dtree_score:.4f}')

Precision del Arbol de Decision (Validacion): 0.8118
Precision del Arbol de Decision (Prueba): 0.7854


**LogisticRegression**

In [25]:
features = df.drop(['is_ultra'],axis=1)
target = df['is_ultra']

#Dividimos en conjuntos
features_train_val, features_test, target_train_val, target_test = train_test_split(features, target, random_state=54321,test_size=0.20)
features_train, features_valid, target_train, target_valid = train_test_split(features_train_val, target_train_val, random_state=54321,test_size=0.25)

model_lr = LogisticRegression(random_state = 54321, solver='liblinear')# inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model_lr.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model_lr.score(features_train, target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model_lr.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación

print(f'Precision de la Regresion Logistica (Prueba): {score_train:.4f}')
print(f'Precision de la Regresion Logistica (Validacion): {score_valid:.4f}')

Precision de la Regresion Logistica (Prueba): 0.7173
Precision de la Regresion Logistica (Validacion): 0.7325


**Resultados**<br>
El modelo que muestra un mayor porcentaje de exactitud es el de RandomForestClassifier, mostrando un 82% de precision en el modelo de Validacion y 78% en el modelo de prueba, cabe mencionar que se probaron diferentes estimadores y profundidades, mismos que indicaron el mas alto como se menciona.
A este mismo modelo, se le aplico la prueba de cordura  (Dummy Classifier), misma que arrojo una precision del 66%

El modelo DecissionTreeClassifier mostro un porcentaje de precision de 81% en el modelo de validacion y 78% en de prueba, por debajo de RandomForest, de igual forma se utilizaron diferentes profundidades antes de conocer el valor mas alto.<br>

Por ultimo el modelo de LogisticRegression, fue el mas bajo, con un nivel de precision en el conjunto de validacion con un 71% y en el conjunto de entrenamiento de 73%.